In [1]:
import pandas as pd
import datetime as dt
from logger_settings import logger

pd.options.display.max_colwidth = 10
pd.options.display.max_rows = None

In [2]:
ticks_columns = {
    'last_price': float,
    'last_traded_quantity': int,
    'total_buy_quantity': int,
    'total_sell_quantity': int,
    'last_trade_time': 'datetime64[ns]',
    'oi': int,
}
tdf = pd.read_json('ticks-formatted.json', dtype=ticks_columns, convert_dates={'last_trade_time': '%Y-%m-%d %H:%M:%S'})
columns_to_drop = set(tdf.columns).difference(ticks_columns.keys())
tdf.drop(columns=columns_to_drop, inplace=True)
tdf.set_index('last_trade_time', inplace=True, drop=False)

In [4]:
START_TIME = dt.datetime.now().replace(year=2024, month=3, day=1)

class Tick:
    def __init__(self, last_price, last_traded_quantity, total_buy_quantity, total_sell_quantity, last_trade_time, oi):
        self.last_price = last_price
        self.last_traded_quantity = last_traded_quantity
        self.total_buy_quantity = total_buy_quantity
        self.total_sell_quantity = total_sell_quantity
        self.last_trade_time = last_trade_time
        self.oi = oi
        self.id = (last_trade_time - START_TIME).seconds

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        return f"Tick: {self.id} {round(self.last_price, 2)}"

class Direction:
    UP = 'up'
    DOWN = 'down'

class Instrument:
    def __init__(self, name):
        self.name = name

class Phase:
    TYPE_CONT = "continuation"
    TYPE_STAG = "stagnation"

    def __init__(self, phase_type, direction, start):
        self.phase_type = phase_type
        self.direction = direction
        self.start = start
        self.end = None
        self.strength = None

class PhaseStartedException(Exception):
    pass

class PhaseSoftRetraceException(Exception):
    pass

class PhaseHardRetraceException(Exception):
    pass

class PhaseTerminatedException(Exception):
    pass

class Settings:
    MIN_UP_PC_CHG = 0.5 / 100 # .5%
    MIN_DOWN_PC_CHG = -0.5 / 100 # -.5%
    BREACH_PC = 5 / 100 # 5% Percentage of continuation phase gains required to breach the trend
    LAST_TICKS_BREACH = {Direction.UP: 5 * 1000, Direction.DOWN: 5 * 1000}
    QUANTITY = 50
    CONFIRM_TICKS = 5 # 5 Seconds


class Order:
    ORDER_ID = 0

    TYPE_BUY = 'buy'
    TYPE_SELL = 'sell'
    STATUS_CREATED = 'CREATED'
    STATUS_EXECUTED = 'EXECUTED'
    STATUS_DECLINE = 'DECLINED'
    STATUS_INTRADE = 'INTRADE'
    STATUS_CLOSED = 'CLOSED'

    def __init__(self, type, limit_price, quantity):
        self.type = type
        self.limit_price = limit_price
        self.quantity = quantity
        self.STATUS = 'CREATED'
        self.square_off_price = None
        self.id = Order.ORDER_ID
        Order.ORDER_ID += 1

    def square_off(self, price):
        self.square_off_price = price
        self.STATUS = Order.STATUS_CLOSED

    @property
    def pnl(self):
        profit = round(self.square_off_price - self.limit_price,  2)
        if self.type == Order.TYPE_BUY:
            return profit
        return - profit

class PhaseOrder:
    def __init__(self, phase, order):
        self.phase = phase
        self.order = order

In [11]:
from collections import deque


class Candle(deque):
    def __init__(self, iterable, *args, **kwargs):
        super().__init__(iterable, *args, **kwargs)
        self.OPEN = iterable.last_price if iterable else None
        self.HIGH = self.OPEN
        self.LOW = self.OPEN
        self.CLOSE = self.OPEN
        self.PERIOD = 0
        self.IS_RED = None

    def append(self, x):
        super().append(x)
        self.HIGH = max(x.last_price, self.HIGH)
        self.LOW = min(x.last_price, self.LOW)
        self.CLOSE = x.last_price
        self.PERIOD = self[-1].id - self[0].id
        self.IS_RED = self.OPEN > self.CLOSE

    def popleft(self):
        if len(self) < Settings.CONFIRM_TICKS:
            raise Exception("not full yet")
        x = super().popleft()
        if self.HIGH == x.last_price:
            self.HIGH = max(self, key=lambda k: k.last_price)
        if self.LOW == x.last_price:
            self.LOW = min(self, key=lambda k: k.last_price)
        self.OPEN = self[0].last_price
        self.PERIOD = self[-1].id - self[0].id
        self.IS_RED = self.OPEN > self.CLOSE
        return x


class PhaseStartFailed(Exception):
    pass

class ContinuationPhase(Phase):
    STATUS_INITIATED = 'INITIATED'
    STATUS_STARTED = 'STARTED'
    STATUS_TERM = 'TERMINATED'
    STATUS_ONG = 'ONGOING'
    STATUS_SOFT_RETR = 'SOFT RETRACING'
    STATUS_HARD_RETR = 'HARD RETRACING'

    def __init__(self, ticks, pm):
        self.phase_type = Phase.TYPE_CONT
        self.t_start = ticks[0]
        self.t_end = ticks[-1]
        self.status = self.STATUS_INITIATED
        self.min_pc = None
        self.direction = None
        self.pm = pm
        self.last_processed_tid = ticks[-1].id
        self.started_at = None
        self.terminated_at = None
        self.hard_retraced_at = None
        self.soft_retraced_at = None
        self.last_5sec = Candle(ticks)
        self.second_last_5sec = Candle([])

    def start_phase(self):
        if len(self.second_last_5sec.PERIOD) < Settings.CONFIRM_TICKS or len(self.last_5sec) == 0:
            return False
        is_red_breach = self.second_last_5sec.IS_RED and self.last_5sec.CLOSE >= self.second_last_5sec.OPEN
        is_green_breach = (not self.second_last_5sec.IS_RED) and self.last_5sec.CLOSE <= self.second_last_5sec.OPEN
        if is_red_breach or is_green_breach:
            raise PhaseStartFailed(f'failed to start phase at: {self.last_5sec.id}')
        price_change = self.last_5sec.CLOSE - self.second_last_5sec.CLOSE
        self.min_pc = Settings.MIN_UP_PC_CHG if price_change >= 0 else Settings.MIN_DOWN_PC_CHG
        self.direction = Direction.UP if price_change >= 0 else Direction.DOWN
        self.status = self.STATUS_STARTED
        self.started_at = tick
        return True

    def __repr__(self):
        return f"Continuation: {self.direction}, {self.status} from {self.t_start} to {self.t_end}, terminated at: {self.terminated_at}, hard retracel: {self.hard_retraced_at}, soft: {self.soft_retraced_at}"

    def __str__(self):
        return self.__repr__()

    def get_last_n_high(self):
        """
        """
        if len(self.pm.ticks) < 2:
            raise Exception(f"insufficient length to calculate last n high len: {len(self.pm.ticks)}")
        if self.direction == Direction.UP:
            return min(self.pm.ticks[-2-Settings.LAST_TICKS_BREACH[self.direction]: -2], key=lambda x: x.last_price).last_price
        elif self.direction == Direction.DOWN:
            return max(self.pm.ticks[-2-Settings.LAST_TICKS_BREACH[self.direction]: -2], key=lambda x: x.last_price).last_price
        else:
            raise NotImplementedError

    def get_last_nsec_ticks(self, n):
        ticks = []
        close_id = self.last_5sec[-1].id
        idx_sl = 0 # index in second last
        start_id = self.second_last_5sec[idx_sl].id
        for idx_sl in range(len(self.second_last_5sec)):
            if close_id - self.second_last_5sec[idx_sl].id <= n:
                ticks.append(self.second_last_5sec[idx_sl])
        for idx_l in range(len(self.last_5sec)):
            ticks.append(self.last_5sec[idx_l])
        return ticks

    def update_last_nsec(self, tick):
        """
        Updates the state for last_5sec and second_last_5sec when new ticks arrive
        """
        while tick.id - self.last_5sec[0].id >= Settings.CONFIRM_TICKS:
            x = self.last_5sec.popleft()
            while x.id - self.second_last_5sec.id >= Settings.CONFIRM_TICKS:
                self.second_last_5sec.popleft()
            self.second_last_5sec.append(x)
        self.last_5sec.append(tick)

    def process(self, tick):
        """
        if it breaks previous open or previous nth tick
        """
        if tick.id <= self.last_processed_tid:
            return
        self.last_processed_tid = tick.id
        self.update_last_nsec(tick)
        if self.status == self.STATUS_INITIATED:
            has_started = self.start_phase()
            if not has_started:
                return
            self.pm.on_phase_start(self, tick)
            change = tick.last_price - self.t_start.last_price
        elif self.status == self.STATUS_TERM or self.status == self.STATUS_HARD_RETR:
            return
        else:
            change = tick.last_price - self.t_end.last_price
        if abs(change) < abs(self.min_pc):
            logger.debug(f"not a prominent change: {tick}, {tick.last_price}")
            return
        
        if self.status in [self.STATUS_STARTED, self.STATUS_ONG, self.STATUS_SOFT_RETR]:
            if (self.direction == Direction.UP and change < 0) or (self.direction == Direction.DOWN and change > 0):
                if (self.direction == Direction.UP and tick.last_price < self.t_start.last_price) or (self.direction == Direction.DOWN and tick.last_price > self.t_start.last_price):
                    self.status = self.STATUS_TERM
                    self.terminated_at = tick
                    if self.hard_retraced_at is None:
                        self.hard_retraced_at = tick
                    if self.soft_retraced_at is None:
                        self.soft_retraced_at = tick
                    raise PhaseTerminatedException(f"Continuation Phase {self} terminated at: {tick}")
                else:
                    last = self.get_last_n_high()
                    if (tick.last_price < last and self.direction == Direction.UP) or (tick.last_price > last and self.direction == Direction.DOWN):
                        self.status = self.STATUS_HARD_RETR
                        self.hard_retraced_at = tick
                        if self.soft_retraced_at is None:
                            self.soft_retraced_at = tick
                        raise PhaseHardRetraceException(f"Continuation Phase {self} hard retracing at: {tick}")
                    else:
                        self.status = self.STATUS_SOFT_RETR
                        self.soft_retraced_at = tick
                        raise PhaseSoftRetraceException(f"Continuation Phase {self} soft retracing at: {tick}")
            elif self.status == self.STATUS_SOFT_RETR:
                self.soft_retraced_at = None
            self.status = self.STATUS_ONG
            self.t_end = tick
        elif self.status == self.STATUS_SOFT_RETR:
            pass
        else:
            raise NotImplementedError


IndentationError: expected an indented block after class definition on line 36 (764781042.py, line 37)

In [6]:
class PhaseManager:
    def __init__(self, instrument):
        self.instrument = instrument
        self.current_phase = None
        self.ps = [] # phase stack
        self.aps = [] # archived phase stack
        self.ticks = []
        self.first_pc = None
        self.current_id = 0
        self.current_order = None
        self.closed_orders = []

    def get_pc_tick(self, tick):
        if len(self.ticks) == 0:
            self.first_pc = tick
        pc_tick = {
            'last_price': (tick['last_price'] - self.first_pc['last_price']) * 100 / self.first_pc['last_price'],
            'last_traded_quantity': (tick['last_traded_quantity'] - self.first_pc['last_traded_quantity']) * 100 / self.first_pc['last_traded_quantity'],
            'total_buy_quantity': (tick['total_buy_quantity'] - self.first_pc['total_buy_quantity']) * 100 / self.first_pc['total_buy_quantity'],
            'total_sell_quantity': (tick['total_sell_quantity'] - self.first_pc['total_sell_quantity']) * 100 / self.first_pc['total_sell_quantity'],
            'last_trade_time': tick['last_trade_time'],
            'oi': (tick['oi'] - self.first_pc['oi']) * 100 / self.first_pc['oi'],
        }
        tick_obj = Tick(**pc_tick)
        self.current_id += 1
        return tick_obj

    def next(self, tick):
        pc_tick = self.get_pc_tick(tick)
        self.ticks.append(pc_tick)
        return self.process(pc_tick)

    def process(self, tick):
        if self.current_phase is None:
            # First call
            self.current_phase = ContinuationPhase(t_start=tick, pm=self)
            return
        # Has existing phase
        try:
            self.current_phase.process(tick)
            self.on_ongoing(self.current_phase, tick)
        except PhaseSoftRetraceException:
            self.on_soft_retracel(self.current_phase, tick)
        except PhaseHardRetraceException:
            self.on_hard_retracel(self.current_phase, tick)
            self.ps.append(self.current_phase)
            ticks = self.current_phase.get_last_nsec_ticks(Settings.CONFIRM_TICKS)
            self.current_phase = ContinuationPhase(ticks=ticks, pm=self)
        except PhaseTerminatedException:
            self.on_hard_retracel(self.current_phase, tick)
            self.on_termination(self.current_phase, tick)
            self.aps.append(self.current_phase)
            ticks = self.current_phase.get_last_nsec_ticks(Settings.CONFIRM_TICKS)
            self.current_phase = ContinuationPhase(ticks=ticks, pm=self)
        except PhaseStartFailed:
            ticks = self.current_phase.get_last_nsec_ticks(Settings.CONFIRM_TICKS)
            self.current_phase = ContinuationPhase(ticks=ticks, pm=self)
        self.process_ps(tick)

    def process_ps(self, tick):
        # Process for process stack
        mark_for_removal = []
        for i in range(len(self.ps)):
            phase = self.ps[i]
            try:
                phase.process(tick)
            except PhaseSoftRetraceException:
                # logger.info(f"soft retracel of {phase}")
                pass
            except PhaseHardRetraceException:
                pass
            except PhaseTerminatedException:
                mark_for_removal.append(i)
                self.aps.append(phase)
        mark_for_removal.reverse()
        for mark in mark_for_removal:
            del self.ps[mark]

    def on_soft_retracel(self, phase, tick):
        # logger.info(f"soft retracel {phase}")
        pass

    def on_phase_start(self, phase, tick):
        self.current_order = Order(type=Order.TYPE_BUY, limit_price=tick.last_price, quantity=Settings.QUANTITY)

    def on_hard_retracel(self, phase, tick):
        if phase.direction == Direction.UP:
            if self.current_order is not None:
                self.current_order.square_off(tick.last_price)
                self.closed_orders.append(PhaseOrder(phase, self.current_order))
                self.current_order = None
            logger.info(f"hard retracel {phase}, Up change %: {phase.started_at.last_price - phase.hard_retraced_at.last_price}")
        else:
            logger.info(f"hard retracel {phase}, Down change %: {phase.started_at.last_price - phase.hard_retraced_at.last_price}")

    def on_termination(self, phase, tick):
        # logger.info(f"termination {phase}")
        pass

    def on_initiation(self, phase, tick):
        # logger.info(f"initiation {phase}")
        pass

    def on_ongoing(self, phase, tick):
        pass
        # logger.info(f"ongoing {phase}")

instrument = Instrument(name="NIFTY 22650 CALL 7 Mar 2024")
pm = PhaseManager(instrument=instrument)
# for i in range(70):
for i in range(tdf.shape[0]):
    pm.next(tdf.iloc[i].to_dict())


print(pm.current_phase)
# print(pm.ps)
# print(pm.aps)

2024-03-11 03:10:06,788 - INFO - 1776230044.py:89 - [140049177486592] - 
hard retracel Continuation: up, TERMINATED from Tick: 39992 0.0 to Tick: 39994 0.58, terminated at: Tick: 39995 -0.58, hard retracel: Tick: 39995 -0.58, soft: Tick: 39995 -0.58, Up change %: 1.1627906976744145
2024-03-11 03:10:06,798 - INFO - 1776230044.py:91 - [140049177486592] - 
hard retracel Continuation: down, TERMINATED from Tick: 39994 0.0 to Tick: 40000 -3.49, terminated at: Tick: 40017 1.16, hard retracel: Tick: 40017 1.16, soft: Tick: 40016 -0.58, Down change %: -1.7441860465116115
2024-03-11 03:10:06,806 - INFO - 1776230044.py:89 - [140049177486592] - 
hard retracel Continuation: up, TERMINATED from Tick: 40016 -0.58 to Tick: 40025 3.49, terminated at: Tick: 40036 -2.33, hard retracel: Tick: 40036 -2.33, soft: Tick: 40034 1.16, Up change %: 3.4883720930232434
2024-03-11 03:10:06,887 - INFO - 1776230044.py:91 - [140049177486592] - 
hard retracel Continuation: down, TERMINATED from Tick: 40034 0.58 to Tic

Continuation: up, SOFT RETRACING from Tick: 40810 0.0 to Tick: 41486 136.63, terminated at: None, hard retracel: None, soft: Tick: 44400 55.23


In [57]:
num = 1
print(pm.closed_orders[num].phase.t_start.id)
print(pm.closed_orders[num].order.limit_price)
print(pm.closed_orders[num].phase.started_at.last_price)
print(pm.closed_orders[num].phase.started_at.id)
print(pm.closed_orders[num].order.square_off_price)
print(pm.closed_orders[num].phase.hard_retraced_at.last_price)
print(pm.closed_orders[num].phase.hard_retraced_at.id)


11
-1.7441860465116321
-1.7441860465116321
12
-2.325581395348829
-2.325581395348829
16


In [82]:
profits = [po.order.pnl for po in pm.closed_orders]
# profits = [round(ph.hard_retraced_at.last_price - ph.started_at.last_price, 2) for ph in pm.aps if ph.direction == Direction.UP]
# profits = [round(ph.hard_retraced_at.last_price - ph.started_at.last_price, 2) for ph in pm.ps if ph.direction == Direction.UP]
print(sum(profits))
print(len(profits))

-17.44
8


In [93]:
pm.ps

[]

In [76]:
pm.aps

[Continuation: up, TERMINATED from Tick: 11767 0.0 to Tick: 11769 0.58, terminated at: Tick: 11770 -0.58, hard retracel: Tick: 11770 -0.58, soft: Tick: 11770 -0.58,
 Continuation: down, TERMINATED from Tick: 11769 0.0 to Tick: 11775 -3.49, terminated at: Tick: 11792 1.16, hard retracel: Tick: 11792 1.16, soft: Tick: 11791 -0.58,
 Continuation: up, TERMINATED from Tick: 11791 -0.58 to Tick: 11800 3.49, terminated at: Tick: 11811 -2.33, hard retracel: Tick: 11811 -2.33, soft: Tick: 11809 1.16,
 Continuation: down, TERMINATED from Tick: 11809 0.58 to Tick: 11853 -16.86, terminated at: Tick: 12048 1.16, hard retracel: Tick: 12048 1.16, soft: Tick: 12047 0.0,
 Continuation: up, TERMINATED from Tick: 12047 0.0 to Tick: 12048 1.16, terminated at: Tick: 12053 -0.58, hard retracel: Tick: 12053 -0.58, soft: Tick: 12049 0.0,
 Continuation: up, TERMINATED from Tick: 12051 -1.16 to Tick: 12053 -0.58, terminated at: Tick: 12054 -1.74, hard retracel: Tick: 12054 -1.74, soft: Tick: 12054 -1.74,
 Conti

In [7]:
# Losses
# multi_y = [[po.order.limit_price, po.order.square_off_price] for po in pm.closed_orders if po.order.pnl <= 0]
# multi_x = [[po.phase.started_at.id, po.phase.hard_retraced_at.id] for po in pm.closed_orders if po.order.pnl <= 0]
# texts = [str(po.order.pnl) for po in pm.closed_orders if po.order.pnl <= 0]
# subplot = {'xs': multi_x, 'ys': multi_y, 'texts': texts, 'x': [m[1] for m in multi_x], 'y': [m[1] for m in multi_y]}

# All
multi_y = [[po.order.limit_price, po.order.square_off_price] for po in pm.closed_orders]
multi_x = [[po.phase.started_at.id, po.phase.hard_retraced_at.id] for po in pm.closed_orders]
texts = [str(po.order.pnl) for po in pm.closed_orders]
subplot = {'xs': multi_x, 'ys': multi_y, 'texts': texts, 'x': [m[1] for m in multi_x], 'y': [m[1] for m in multi_y]}

from bokeh.models import ColumnDataSource
subplot = ColumnDataSource(subplot)

In [88]:
pm.current_phase

Continuation: up, SOFT RETRACING from Tick: 12585 0.0 to Tick: 13261 136.63, terminated at: None, hard retracel: None, soft: Tick: 16175 55.23

In [8]:
import utils as ut
# ut.bokeh_plot(x=list(range(tdf.shape[0])), y=tdf.last_price, x_label="timestamp", y_label="price", plot='line')
ut.bokeh_plot(x=[m.id for m in pm.ticks], y=[tick.last_price for tick in pm.ticks], x_label="timestamp", y_label="price", plot='line', subplots=subplot)
# tdf

Loading BokehJS ...

In [9]:
multi_y = [[ph.started_at.last_price, ph.hard_retraced_at.last_price] for ph in pm.aps if ph.direction == Direction.UP]
multi_x = [[ph.started_at.id, ph.hard_retraced_at.id] for ph in pm.aps if ph.direction == Direction.UP]
texts = [str(round(ph.hard_retraced_at.last_price - ph.started_at.last_price, 2)) for ph in pm.aps if ph.direction == Direction.UP]
subplot = {'xs': multi_x, 'ys': multi_y, 'texts': texts, 'x': [m[1] for m in multi_x], 'y': [m[1] for m in multi_y]}

from bokeh.models import ColumnDataSource
subplot = ColumnDataSource(subplot)